<a href="https://colab.research.google.com/github/Granero0011/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/LS_DS_Unit_4_Sprint_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Perceptron on XOR Gates](#Q2)
3. [Multilayer Perceptron](#Q3)
4. [Keras MMP](#Q4)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:**
- **Input Layer:**
- **Hidden Layer:**
- **Output Layer:**
- **Activation:**
- **Backpropagation:**


In [0]:
### Answering the questions
''' Neuron: A neuron is a basic unit of a neural network
    Input layer: An input layer is the first layer, which receives the the source data
    Hidden layer: layer in between input layers and output layers, where artificial neurons take in a set of weighted inputs and produce an output through an activation function 
    Output layer: layer that produces the output 
    Activation: defines the output by applying a function to the input
    Backpropagation: Mechanism by which neural networks learn'''

## 2. Perceptron on XOR Gates <a id="Q3=2"></a>

Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

|x1	|x2|x3|	y|
|---|---|---|---|
1|	1|	1|	1|
1|	0|	1|	0|
0|	1|	1|	0|
0|	0|	1|	0|

In [0]:
#Let's load the libraries
import numpy as np
import pandas as pd

In [0]:
#Perceptron class 
class Perceptron(object):
    """Implements a perceptron network"""
    def __init__(self, input_size, lr=1, epochs=100):
        self.W = np.zeros(input_size+1)
        # add one for bias
        self.epochs = epochs
        self.lr = lr
    
    def activation_fn(self, x):
        #return (x >= 0).astype(np.float32)
        return 1 if x >= 0 else 0
 
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [0]:
#Let's create the example
X = np.array([[1,1,1],[1,0,1],[0,1,1],[0,0,1]])
d=np.array([1, 0, 0, 0])

In [6]:
#Let's check the weights
perceptron = Perceptron(input_size=3)
perceptron.fit(X, d)
print(perceptron.W)

[-2.  1.  3. -2.]


## 3. Multilayer Perceptron <a id="Q3"></a>

Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights.
Your network must have one hidden layer.
You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
Train your model on the Heart Disease dataset from UCI:



In [0]:
heart = pd.read_csv('https://raw.githubusercontent.com/Granero0011/DS-Unit-4-Sprint-2-Neural-Networks/master/heart.csv')

In [11]:
heart.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [12]:
heart.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [0]:
features= heart.columns.tolist()

In [0]:
features.remove('target')

In [0]:
target='target'

In [0]:
X= heart[features]
y=heart[target]

In [0]:
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher

class NeuralNetwork: 
    def __init__(self):
        # Set upArchietecture 
        self.inputs = 13
        self.hiddenNodes = 2
        self.outputNodes = 1
        
        #Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes) #2x3
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes) #3x1
    
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        #Weighted sume of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        #Acivations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [30]:

nn = NeuralNetwork()

for i in range(1000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 50 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
      age  sex  cp  trestbps  chol  ...  exang  oldpeak  slope  ca  thal
0     63    1   3       145   233  ...      0      2.3      0   0     1
1     37    1   2       130   250  ...      0      3.5      0   0     2
2     41    0   1       130   204  ...      0      1.4      2   0     2
3     56    1   1       120   236  ...      0      0.8      2   0     2
4     57    0   0       120   354  ...      1      0.6      2   0     2
5     57    1   0       140   192  ...      0      0.4      1   0     1
6     56    0   1       140   294  ...      0      1.3      1   0     2
7     44    1   1       120   263  ...      0      0.0      2   0     3
8     52    1   2       172   199  ...      0      0.5      2   0     3
9     57    1   2       150   168  ...      0      1.6      2   0     2
10    54    1   0       140   239  ...      0      1.2      2   0     2
11    48    0   2       130   275  ...      0      0.2      2   0     2
12    49    1   1       130

Exception: ignored

## 4. Keras MMP <a id="Q4"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [0]:

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [33]:
model = Sequential()

W0722 03:34:30.534475 139978841569152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [34]:

model.add(Dense(1, input_dim=13, activation="sigmoid"))

W0722 03:35:56.818556 139978841569152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 03:35:56.832584 139978841569152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0722 03:38:27.353671 139978841569152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 03:38:27.383052 139978841569152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0722 03:38:27.389034 139978841569152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [37]:
model.fit(X, y, epochs=150)

W0722 03:39:28.358780 139978841569152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/150
303/303 [==============================] - 1s 2ms/step - loss: 8.7772 - acc: 0.4554
Epoch 2/150
303/303 [==============================] - 0s 41us/step - loss: 8.7772 - acc: 0.4554
Epoch 3/150
303/303 [==============================] - 0s 41us/step - loss: 8.7772 - acc: 0.4554
Epoch 4/150
303/303 [==============================] - 0s 50us/step - loss: 8.7772 - acc: 0.4554
Epoch 5/150
303/303 [==============================] - 0s 56us/step - loss: 8.7772 - acc: 0.4554
Epoch 6/150
303/303 [==============================] - 0s 47us/step - loss: 8.7772 - acc: 0.4554
Epoch 7/150
303/303 [==============================] - 0s 46us/step - loss: 8.7772 - acc: 0.4554
Epoch 8/150
303/303 [==============================] - 0s 45us/step - loss: 8.7772 - acc: 0.4554
Epoch 9/150
303/303 [==============================] - 0s 47us/step - loss: 8.7772 - acc: 0.4554
Epoch 10/150
303/303 [==============================] - 0s 48us/step - loss: 8.7772 - acc: 0.4554
Epoch 11/150
303/303 [========

In [41]:

scores = model.evaluate(X, y)
print(f"{model.metrics_names[1]}: {scores[1]*100}")

303/303 [==============================] - 0s 99us/step
acc: 45.54455445544555


In [46]:
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=13, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/20
202/202 [==============================] - 0s 1ms/step - loss: 4.9808 - acc: 0.6832
Epoch 2/20
202/202 [==============================] - 0s 135us/step - loss: 4.5432 - acc: 0.6881
Epoch 3/20
202/202 [==============================] - 0s 140us/step - loss: 2.6255 - acc: 0.6782
Epoch 4/20
202/202 [==============================] - 0s 137us/step - loss: 1.3802 - acc: 0.6238
Epoch 5/20
202/202 [==============================] - 0s 133us/step - loss: 1.1582 - acc: 0.6485
Epoch 6/20
202/202 [==============================] - 0s 145us/step - loss: 0.9586 - acc: 0.6535
Epoch 7/20
202/202 [==============================] - 0s 120us/step - loss: 0.8324 - acc: 0.6337
Epoch 8/20
202/202 [==============================] - 0s 120us/step - loss: 0.7491 - acc: 0.6337
Epoch 9/20
202/202 [==============================] - 0s 148us/step - loss: 0.7123 - acc: 0.6436
Epoch 10/20
202/202 [==============================] - 0s 135us/step - loss: 0.6667 - acc: 0.6436
Epoch 11/20
202/202 [==========

In [47]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.5445544585929846 using {'batch_size': 40, 'epochs': 20}
Means: 0.32673267685737545, Stdev: 0.27969201782181613 with: {'batch_size': 10, 'epochs': 20}
Means: 0.38943894718936567, Stdev: 0.20647514509514603 with: {'batch_size': 20, 'epochs': 20}
Means: 0.5445544585929846, Stdev: 0.41308215433215534 with: {'batch_size': 40, 'epochs': 20}
Means: 0.36633663012249634, Stdev: 0.26158108414504844 with: {'batch_size': 60, 'epochs': 20}
Means: 0.21122112374740465, Stdev: 0.2917387233791075 with: {'batch_size': 80, 'epochs': 20}
Means: 0.5445544507243846, Stdev: 0.4130821526347633 with: {'batch_size': 100, 'epochs': 20}
